<!-- PROJECT LOGO -->
<br />
<div align="center">
  <a>
    <img src="https://res.cloudinary.com/dek4evg4t/image/upload/v1729273000/Group_4.png" alt="Logo" width="30%">
  </a>
</div>

### 🖹 Descripción:
El Proyecto II tiene como objetivo aplicar redes neuronales convolucionales (CNN) para realizar una clasificación multiclase de imágenes mediante aprendizaje supervisado. Utilizando el [Covid-19 Image Dataset de Kaggle](https://www.kaggle.com/datasets/pranavraikokte/covid19-image-dataset), que contiene imágenes de rayos X clasificadas en tres categorías (Covid-19, Normal, Neumonía), en este proyecto se desarrollarán clasificadores capaces de diagnosticar enfermedades pulmonares. El proyecto también explora el uso de PyTorch para el desarrollo de modelos de Machine Learning y herramientas de monitoreo, como Weights and Biases, para el seguimiento en tiempo real del proceso de entrenamiento.

### 👣 Pasos a seguir:

1. Importación de librerías
2. Configuraciones Iniciales
3. Preparación de datos
4. Definición del modelo
5. Ajuste de hiperparámetros
6. Entrenamiento del modelo
7. Evaluación del modelo

### ✍️ Autores:
* Angelo Ortiz Vega - [@angelortizv](https://github.com/angelortizv)
* Alejandro Campos Abarca - [@MajinLoop](https://github.com/MajinLoop)

### 📅 Fecha:
20 de octubre de 2024

### 📝 Notas:
Este es el segundo proyecto del curso IC6200 - Inteligencia Artificial. En este notebook, titulado "Covid-19 Classification", se profundiza en técnicas de data augmentation, preprocesamiento de imágenes con filtros, y fine-tuning de modelos CNN como VGG16 para mejorar la capacidad de generalización de las redes neuronales convolucionales.

### Otras notas:
Asegurarse de contar con Python y las siguientes bibliotecas instaladas: torch, torchvision, cv2, numpy, matplotlib, Pillow.

# Paso 1: Importación de librerías

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import wandb
from sklearn.metrics import accuracy_score

# Paso 2: Configuraciones Iniciales

## 2.1. Verificación de estado de CUDA

In [17]:
u.check_cuda_info()

CUDA available: True
CUDA version: 12.1
Number of GPUs: 1
Current GPU: 0


## 2.2. Configuración de Weights & Biases

In [20]:
wandb.init(project="Covid19-CNN-Classification", entity="angelortizv-tecnologico-de-costa-rica") 
wandb.config = {
    "learning_rate": 0.001,
    "batch_size": 32,
    "epochs": 10
}

# Paso 3: Preparación de Datos

In [22]:
def get_data_loaders(batch_size):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

    train_dataset = datasets.ImageFolder('data/Covid19-dataset/train', transform=transform)
    test_dataset = datasets.ImageFolder('data/Covid19-dataset/test', transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# Cargar datos
train_loader, test_loader = get_data_loaders(wandb.config["batch_size"])

# Paso 4: Definición del Modelo

In [30]:
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT  # Puedes usar IMAGENET1K_V1 si lo prefieres
model = models.vgg16(weights=weights)
model.classifier[6] = nn.Linear(4096, 3)  # Cambiar la salida a 3 clases
model = model.to(device)

# Paso 5: Ajuste de Hiperparámetros

In [31]:
import optuna

def objective(trial):
    # Hiperparámetros a ajustar
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    num_epochs = 10

    # Cargar datos
    train_loader, test_loader = get_data_loaders(batch_size)

    # Reiniciar el modelo
    model = models.vgg16(pretrained=True)
    model.classifier[6] = nn.Linear(4096, 3)  # Cambiar la salida a 3 clases
    model = model.to(device)

    # Definir el optimizador y la función de pérdida
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Entrenamiento
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Reiniciar gradientes
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass y optimización
            loss.backward()
            optimizer.step()

    # Evaluar el modelo
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    wandb.log({"accuracy": accuracy})
    return accuracy

# Ejecutar Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros:")
print(study.best_params)
print("Mejor precisión:", study.best_value)

[I 2024-10-18 19:26:33,232] A new study created in memory with name: no-name-be2cd131-a226-428c-968f-d187a259eab4
C:\Users\PC MASTER\AppData\Local\Temp\ipykernel_35228\1212401057.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
C:\Users\PC MASTER\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\PC MASTER\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to 

Mejores hiperparámetros:
{'batch_size': 32, 'learning_rate': 2.6853468906306184e-05}
Mejor precisión: 1.0


# Paso 6: Entrenamiento del Modelo

In [32]:
# Paso 6: Entrenamiento del modelo
# Utiliza el mejor conjunto de hiperparámetros encontrados
best_params = study.best_params
batch_size = best_params['batch_size']
learning_rate = best_params['learning_rate']

# Reentrenar el modelo con los mejores hiperparámetros
train_loader, test_loader = get_data_loaders(batch_size)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(wandb.config.epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    wandb.log({"loss": loss.item(), "epoch": epoch})


AttributeError: 'dict' object has no attribute 'epochs'

# Paso 7: Evaluación del Modelo

In [ ]:
# Paso 7: Evaluación del modelo
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Precisión del modelo: {accuracy * 100:.2f}%")
wandb.log({"final_accuracy": accuracy})
